- use Azure Cosmos NoSQL as Vector DB

In [ ]:
%pip install -U langchain_openai

In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
%pip install langchain azure-cosmos langchain_openai langchain-community pypdf azure-identity python-dotenv openai 

#### global vars

In [ ]:
from dotenv import load_dotenv
load_dotenv('/lakehouse/default/Files/.env')

cosmos_endpoint = 'https://cosmos-vector-db-1.documents.azure.com'
azure_openai_key = ''
cosmos_key = ''

StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 43, Finished, Available, Finished)

#### load and split PDF data

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/lakehouse/default/Files/9781509302963_Microsoft Azure Essentials Fundamentals of Azure 2nd ed mobile.pdf")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

docs = text_splitter.split_documents(data)

print(len(docs))


StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 27, Finished, Available, Finished)

787


#### encode documents to embedding

In [6]:
from langchain_openai import AzureOpenAIEmbeddings

azopenai_embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    azure_endpoint="https://aihub-openai-base-model.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15", #If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    api_key=azure_openai_key, # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    openai_api_version='2023-05-15', # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)


def generate_embeddings(text):
    return azopenai_embeddings.embed_query(text)




StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 19, Finished, Available, Finished)

#### setup Cosmos as Vector DB

In [39]:
# https://python.langchain.com/docs/integrations/vectorstores/azure_cosmos_db_no_sql/
# https://github.com/microsoft/AzureDataRetrievalAugmentedGenerationSamples/blob/main/Python/CosmosDB-NoSQL-Integrations/LangChain-CosmosDBNoSQL-AzureOpenAI.ipynb

from azure.identity import DefaultAzureCredential
from azure.cosmos import CosmosClient, PartitionKey

indexing_policy = {
    "indexingMode": "consistent",
    "includedPaths": [{"path": "/*"}],
    "excludedPaths": [{"path": '/"_etag"/?'}],
    "vectorIndexes": [{"path": "/content_embedding", "type": "diskANN"}],
    "fullTextIndexes": [{"path": "/text"}],
}

vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path": "/content_embedding",
            "dataType": "float32",
            "distanceFunction": "cosine",
            "dimensions": 1536,
        }
    ]
}

full_text_policy = {
    "defaultLanguage": "en-US",
    "fullTextPaths": [{"path": "/text", "language": "en-US"}],
}

azcred = DefaultAzureCredential()

cosmos_client = CosmosClient(cosmos_endpoint,  cosmos_key)
database_name = "cosmos-vector-db-1"
container_name = "doc_azure_essential"
partition_key = PartitionKey(path="/id")
cosmos_container_properties = {"partition_key": partition_key}

# create db
database = cosmos_client.create_database_if_not_exists(database_name)
print('Database with id \'{0}\' created'.format(database_name))

# container = database.create_container_if_not_exists(
#     id=container_name,
#     partition_key=PartitionKey(path="/id"),
#     offer_throughput=400,
#     indexing_policy=indexing_policy,
#     vector_embedding_policy=vector_embedding_policy   ,
#     full_text_policy=full_text_policy
# )
print('Container with id \'{0}\' created'.format(container_name))


# content
# content_embedding

StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 52, Finished, Available, Finished)

Database with id 'cosmos-vector-db-1' created
Container with id 'doc_azure_essential' created


In [8]:
# docs_to_load = []

# for idx, doc in enumerate(docs):
#     embedding = generate_embeddings(doc.page_content)
#     docs_to_load.append({
#         # 'id': idx,
#         'text': doc.page_content,
#         'embedding': embedding,
#         'page': doc.metadata['page'],
#         'total_pages': doc.metadata['total_pages'],
#         'title': doc.metadata['title']
#     })

# print(docs_to_load[0])


StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 21, Finished, Available, Finished)

#### upsert docs to Cosmos

In [28]:
from langchain_community.vectorstores.azure_cosmos_db_no_sql import (
    AzureCosmosDBNoSqlVectorSearch
)

# typical way to let langchain insert the documents in Cosmos, but this way do not allw 

# has existing embedding
vector_store = AzureCosmosDBNoSqlVectorSearch(
    embedding=azopenai_embeddings,
    cosmos_client=cosmos_client,
    database_name=database_name,
    container_name=container_name,
    vector_embedding_policy=vector_embedding_policy,
    indexing_policy=indexing_policy,
    full_text_policy=full_text_policy,
    cosmos_database_properties={},
    cosmos_container_properties=cosmos_container_properties,
    create_container=False,
)

# insert docs into Cosmos Vector DB
# vector_store = AzureCosmosDBNoSqlVectorSearch.from_documents(
#     documents=docs,
#     embedding=azopenai_embeddings,
#     cosmos_client=cosmos_client,
#     database_name=database_name,
#     container_name=container_name,
#     vector_embedding_policy=vector_embedding_policy,
#     indexing_policy=indexing_policy,
#     full_text_policy=full_text_policy,
#     cosmos_database_properties={},
#     cosmos_container_properties=cosmos_container_properties,
#     create_container=False,
# )


StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 41, Finished, Available, Finished)

In [24]:


results = container.query_items(
        query= '''
        SELECT TOP @num_results c.text, VectorDistance(c.content_embedding, @embedding) as SimilarityScore 
        FROM doc_azure_essential as c
        WHERE VectorDistance(c.content_embedding,@embedding) > @similarity_score
        ORDER BY VectorDistance(c.content_embedding,@embedding)
        ''',
        parameters=[
            {"name": "@embedding", "value": generate_embeddings('who is the author')},
            {"name": "@num_results", "value": 5},
            {"name": "@similarity_score", "value": 0.5}
        ],
        enable_cross_partition_query=True, populate_query_metrics=True)
results = list(results)

context = '\n\n'.join([x['content'] for x in results])

context

StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 37, Finished, Available, Finished)

''

In [ ]:
try:
    query = "who is the author of the Fundamentals of Azure"

    result = vector_store.similarity_search(query)

    print(result)
except ValueError as e:
    print(e.message)

In [45]:
from langchain_community.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.chat_message_histories import CosmosDBChatMessageHistory

prompt_template = """
You are an upbeat AI assistant who is excited to help answer questions. 
You can use this context

{context},

or this chat history

{chat_history},

to answer this question. 

Question: {question}
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
"""
chatbot_prompt = PromptTemplate(
    template = prompt_template, input_variables = ["context", "question", "chat_history"])


llm = AzureChatOpenAI(
        azure_endpoint = 'https://aihub-openai-base-model.openai.azure.com/',
        api_key = azure_openai_key,
        api_version = '2024-12-01-preview',
        azure_deployment = 'gpt-4.1', 
        cache = False,
        n = 1)

retriever = vector_store.as_retriever(search_type = "similarity",search_kwargs = {"k": 5, 'score_threshold': 0.2})

sem_qa = ConversationalRetrievalChain.from_llm(
        llm = llm,
        chain_type = "stuff",
        retriever = retriever,
        return_source_documents = True,
        combine_docs_chain_kwargs = {"prompt": chatbot_prompt}
)

chatbot_chain_retriever=llm
chatbot_chain_llm=retriever
chatbot_chain=sem_qa


# Clearing Semantic Cache before every testing cycle
# database.delete_container("abstracts_history")

cosmos_message_history = CosmosDBChatMessageHistory(
    session_id = "test_session",
    cosmos_endpoint = cosmos_endpoint,
    cosmos_database = database_name,
    cosmos_container = "abstracts_history",
    credential = cosmos_key,
    user_id = "ak")
cosmos_message_history.prepare_cosmos()

conversational_memory = ConversationBufferMemory(
    chat_memory=cosmos_message_history,
    memory_key='chat_history',
    return_messages=True)

StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 58, Finished, Available, Finished)

In [46]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 59, Finished, Available, Finished)

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 31, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_07e970ab25', 'finish_reason': 'stop', 'logprobs': None}, id='run--a1db190d-e9e0-4f91-bdb7-7d5a9e0ea491-0')

In [51]:
response = chatbot_chain.invoke({"question": 'what is Azure fundamental', "chat_history":conversational_memory.buffer_as_messages[-6:]},temperature=0.2)

print(response['answer'])

StatementMeta(, 5cea67fe-861f-4b4f-9d5b-007f8ee605e1, 64, Finished, Available, Finished)

Azure fundamental refers to the core concepts and basic services of Microsoft Azure—the cloud computing platform from Microsoft. These fundamentals include the essential features and services that most Azure users need to know to get started and to build solutions in the cloud.

According to the context from the book Fundamentals of Azure (Second Edition), Azure fundamentals typically cover the most widely used services and concepts, such as:

- Getting started with Azure and understanding what cloud computing is
- Learning about Azure Resource Manager and Role-Based Access Control
- Creating and managing Virtual Machines (VMs) and virtual networks
- Setting up websites and storage accounts
- Basic networking, security, and management tools

The book focuses on these key services and features because they are considered fundamental by the authors, based on many years of real-world Azure projects. While Azure offers a wide range of services, the fundamentals are those that provide the f

In [ ]:
# response = chatbot_chain.invoke({"question": user_message, "chat_history":conversational_memory.buffer_as_messages[-6:]},temperature=0.2)

# # Append user message and response to chat history
# hist.append(["User: "+user_message, "Chatbot: " + response['answer']])

# cosmos_message_history.add_user_message(user_message)

# cosmos_message_history.add_ai_message(response['answer'])